<a href="https://colab.research.google.com/github/nakamura196/000_tools/blob/main/LlamaIndex%2BGPT4%2Bgradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LlamaIndex+GPT4+gradio

以下の記事を参考にしています。

https://qiita.com/DeepTama/items/1a44ddf6325c2b2cd030

2024年4月20日時点のライブラリで動作するように修正しています。

以下のデータを使用しています。

TEIを用いた『渋沢栄一伝記資料』テキストデータの再構築と活用

https://github.com/shibusawa-dlab/lab1

## 環境変数の準備

以下を参考に、`OPENAI_API_KEY`を設定します。

https://note.com/npaka/n/n79bb63e17685

In [ ]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

## ライブラリのインストール

In [ ]:
# LlmaIndexパッケージのインストール (今回は0.8.0で検証)
!pip install llama-index
!pip install gradio

In [ ]:
# ログレベルの設定
import logging
import sys
from llama_index.core import VectorStoreIndex
from llama_index.core.llms import ChatMessage
from llama_index.llms.openai import OpenAI
from llama_index.readers.file.docs.base import DocxReader
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import GPTVectorStoreIndex, ServiceContext, SimpleDirectoryReader
from llama_index.core.callbacks import CallbackManager, LlamaDebugHandler
from llama_index.core.prompts.prompts import QuestionAnswerPrompt
import glob
from bs4 import BeautifulSoup
import gradio as gr

In [ ]:
# Open AIのGPT4をLLMとして指定しServiceContextに設定する
llm = OpenAI(model='gpt-4', temperature=0.5, max_tokens=1024)
llama_debug_handler = LlamaDebugHandler()
callback_manager = CallbackManager([llama_debug_handler])
service_context = ServiceContext.from_defaults(llm=llm, callback_manager=callback_manager) # , callback_manager=callback_manager

## テキスト

In [ ]:
# 場所は任意ですが、今回は `sample_data` の下に、 `data`ディレクトを作成
%cd sample_data
%mkdir data

In [ ]:
# テキストデータのダウンロード
!wget https://raw.githubusercontent.com/shibusawa-dlab/lab1/master/tei/DKB01.xml
!wget https://raw.githubusercontent.com/shibusawa-dlab/lab1/master/tei/DKB02.xml

In [ ]:
!rm -rf /content/sample_data/data
!mkdir /content/sample_data/data

files = glob.glob("*.xml")

# 各ファイルをループ処理
for file_name in files:
    # ファイルを開いて読み込む
    with open(file_name, 'r', encoding='utf-8') as file:
        # BeautifulSoupオブジェクトを作成
        soup = BeautifulSoup(file, 'lxml-xml')  # XMLファイル用のパーサーを指定

        entries = soup.find_all(type="diary-entry")

        for entry in entries:
          id = entry["xml:id"]

          output_path = f"/content/sample_data/data/{id}.txt"

          with open(output_path, 'w', encoding='utf-8') as output_file:
              text_only = entry.get_text()
              text_only = ' '.join(text_only.split())
              output_file.write(text_only)  # Convert the BeautifulSoup tag object to string


In [ ]:
# ドキュメントの読み込み
documents = SimpleDirectoryReader(
    input_dir="/content/sample_data/data"
).load_data()

In [ ]:
# Document -> Knowledge Base生成を行うステップ
index = GPTVectorStoreIndex.from_documents(documents, service_context=service_context)

## 保存（参考）

In [ ]:
storage_context = index.storage_context
storage_context.persist(persist_dir="./storage_context")

In [ ]:
!zip -r storage_context.zip "./storage_context"

## 実行

In [ ]:
# QAテンプレートの準備
qa_template = QuestionAnswerPrompt("""<s>[INST] <<SYS>>
質問に答えてください。
<</SYS>>
== 以下にコンテキスト情報を提供します。
{context_str}
== 質問
{query_str}

 [/INST]
""")

In [ ]:
similarity_top_k = 30 # 50 # 10 # 20 # 3

In [ ]:
# LLMへの問い合わせ
# この時indexを参照し、問い合わせに近い情報（チャンク）を取得し、それをプロンプトに組み込む
# 幾つのチャンクを組み込むのかを `similarity_top_k` で指定する
query_engine = index.as_query_engine(
    similarity_top_k=similarity_top_k,
    text_qa_template=qa_template,
)

In [ ]:
def show_response(response):
    for node in response.source_nodes:
        if node.node.extra_info is not None:
            if "file_name" in node.node.extra_info:
                print(node.node.extra_info["file_name"])
        print(node.node.node_info)
        if node.score is not None:
            print(node.score)
        print("-----------------------------------------------------------------------------")
        print(node.node.text)

        print("============================================================================================================")

In [ ]:
query = "あなたは何時に起床することが多いですか？"

response = query_engine.query(query)
print(response)

In [ ]:
'''
query = "よく行く場所はどこですか？"

response = query_engine.query(query)
print(response)
'''

In [ ]:
'''
query = "誰と会うことが多いですか？"

response = query_engine.query(query)
print(response)
'''

In [ ]:
query = "京釜鉄道会社事務所はどこにありますか？"
response = query_engine.query(query)
print(response)

In [ ]:
show_response(response)

## Gradio

In [ ]:
def add_text(history, text):
    history = history + [(text, None)]
    return history, gr.Textbox(value="", interactive=False)

def bot(history):
    query = history[-1][0]
    response = str(query_engine.query(query))
    history[-1][1] = ""
    for character in response:
        history[-1][1] += character
        yield history

# show_error=True
with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    with gr.Row():
        txt = gr.Textbox(
            scale=4,
            show_label = False,
            container = False
        )
    clear = gr.Button("Clear")

    txt_msg = txt.submit(add_text, [chatbot, txt], [chatbot, txt], queue = False).then(bot, chatbot, chatbot)
    txt_msg.then(lambda: gr.Textbox(interactive = True), None, [txt], queue = False)
    clear.click(lambda: None, None, chatbot, queue=False)

demo.queue()
demo.launch()